In [1]:
# Most Basic
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import math
from os import path
import sys
from datetime import datetime
from scipy.stats import norm
import scipy.stats as ss
import copy

# Filter Specific Functionalities
from filterpy.kalman import UnscentedKalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints

# Gradient Based
import autograd.numpy as a_np
import autograd.scipy.stats as a_ss
from autograd import grad, multigrad_dict, elementwise_grad, jacobian

# Self Defined functions
from dataCleaning import DataCleaner
from ESN import EchoState

In [33]:
data_months = range(1, 13)
data_years = range(2019, 2021)

topK = 2
data = {}
cleaner = DataCleaner(columns=["OptionType", "expirydate", "date", "strike", "spotclose", "close2", "volume"],
                          date_columns=["date", "expirydate"], verbose=False)
for year in data_years:
    for month in data_months:
        if month < 10:
            name = "SPX_{}0{}_1day".format(year, month)
        else:
            name = "SPX_{}{}_1day".format(year, month)
        if not path.exists("./SPXdata/{}.csv".format(name)):
            continue
        cleaner.insert_data(data, name, "USD3MTD156N", topK)
#         print("Added Year: {}, Month: {} data".format(year, month))

cleaner.get_num_days()
cleaner.get_num_observation()
data[np.datetime64('2019-01-02T00:00:00.000000000')]

Attached 315 days of data
Attached 630 option observations


[     OptionType expirydate       date  strike  spotclose  close2  volume  \
 1265       call 2019-06-21 2019-01-02  3050.0    2510.03    3.55   30005   
 1268       call 2019-06-21 2019-01-02  3300.0    2510.03    0.85   30005   
 
             T  
 1265 170 days  
 1268 170 days  , 0.0279388, 0.06809787234042562]

In [34]:
# Define transition Dynamics
# Positional requirement, no real effect, will be replaced by ESN and BS defined further down
def fx(x, dt):
    return x*dt
def hx(x):
    return x

def sigmoid(x):
    return np.divide(1, 1+np.exp(-x))

def ESNf(theta_now, dt, **fx_args):
    """
    :params theta_now theta_t
    
    **fx_args
    :params G reservoir transition matrix
    :params G_in input translation matrix
    :params u_quad_now u_t^2
    """
    G = fx_args["G"]
    G_in = fx_args["G_in"]
    u_quad_now = fx_args["u_quad_now"]
    theta_next = sigmoid(np.matmul(G, theta_now) + np.matmul(G_in, u_quad_now))
    return theta_next
    
def BSf(theta_t, **hx_args):
    """
    :params theta_t current state vector
    
    **hx_args
    :params p_t current asset price
    :params r_t current asset price
    :params K_t strike price, 1d vector
    :params T_t maturity time, 1d vector
    """
    p_t = hx_args["p_t"]
    r_t = hx_args["r_t"]
    K_t = hx_args["K_t"]
    T_t = hx_args["T_t"]
    dividor = np.sqrt(np.average(theta_t) * T_t)
    d_pls = (np.log(p_t/K_t) + np.multiply(r_t+np.average(theta_t)/2, T_t)) / dividor
    d_mns = (np.log(p_t/K_t) + np.multiply(r_t-np.average(theta_t)/2, T_t)) / dividor
    y_t = np.multiply(p_t, norm.cdf(d_pls)) - np.multiply(np.multiply(K_t, np.exp(-r_t*T_t)), norm.cdf(d_mns))
    return y_t



In [44]:
import autograd.numpy as a_np
import autograd.scipy.stats as a_ss
from filterpy.kalman import MerweScaledSigmaPoints
from math import log, pi


def expectation_term_one(params, dist_mean_t, dist_cov_t):
    """
    E[ theta_t^T W^{-1} theta_t | X]

    :param params: [G, G_in, W, v]
    :param dist_mean_t: E[theta_t | X]
    :param dist_cov_t: Cov[theta_t | X]
    :return: expectation of the first term: E[ theta_t^T W^{-1} theta_t | X]
    """
    G, G_in, W, v = params
    first_term = a_np.trace(a_np.matmul(a_np.linalg.inv(W), dist_cov_t))
    second_term = a_np.matmul(a_np.matmul(dist_mean_t.T, a_np.linalg.inv(W)), dist_mean_t)
    return first_term+second_term


def expectation_term_two(params,
                         dist_mean_t, dist_mean_t_1,
                         dist_cov_t, dist_cov_t_1, dist_cross_t_t_1, u_quad_t):
    """
    E[theta_t^TW^{-1}Phi(theta_{t-1}) | X]

    :param params:              [G, G_in, W, v]
    :param dist_mean_t:         E[theta_t, theta_{t-1} | X]
    :param dist_mean_t_1:       E[theta_t, theta_{t-1} | X]
    :param dist_cov_t:          Cov[theta_t | X]
    :param dist_cov_t_1:        Cov[theta_{t-1} | X]
    :param dist_cross_t_t_1:    P_{t,t-1}^*
    :param u_quad_t:            u_t^2
    :return: expectation of the second term: E[theta_t^TW^{-1} Phi(theta_{t-1}) | X]
    """
    G, G_in, W, v = params
    # Linear transform
    L = dist_mean_t.shape[0]
    trans_mean_t_1 = a_np.matmul(G, dist_mean_t_1) + a_np.matmul(G_in, u_quad_t)    # (L, 1)
    trans_mean = a_np.concatenate((dist_mean_t, trans_mean_t_1), axis=0)            # (2L, 1)
    trans_cross_t_t_1 = a_np.matmul(dist_cross_t_t_1, G.T)                          # (L, L)
    trans_cov_t_1 = a_np.matmul(G, a_np.matmul(dist_cov_t_1, G.T))                  # (L, L)
    first_line = a_np.concatenate((dist_cov_t, trans_cross_t_t_1), axis=1)          # (L, 2L)
    second_line = a_np.concatenate((trans_cross_t_t_1.T, trans_cov_t_1), axis=1)    # (L, 2L)
    trans_cov = a_np.concatenate((first_line, second_line), axis=0)                 # (2L, 2L)

    # sigmoid transform
    dim = 2*L
    SigmaMaker = MerweScaledSigmaPoints(dim, alpha=.1, beta=2., kappa=0)

    sigmas = SigmaMaker.sigma_points(trans_mean, trans_cov)                         # (4L+1, 2L)
    num_sigmas = sigmas.shape[0]
    Wm, Wc = SigmaMaker.Wm, SigmaMaker.Wc                                           # (4L+1, ) (4L+1, )

    def quasi_sigmoid(merged_vec, separate_dim_L):
        """
        :param merged_vec: a merged vector (2L, 1)
        :param separate_dim_L: size of separate vector L
        :return: quasi sigmoid transformed vector (2L, 1)
        """
        upper = merged_vec[:separate_dim_L]
        lower = merged_vec[L:]
        transformed_lower = 1 / (1 + a_np.exp(lower))
        return a_np.concatenate((upper, transformed_lower), axis=0)

    final_mean = a_np.zeros([dim, 1])                                               # (2L, 1)
    for i in num_sigmas:
        final_mean += Wm[i] * quasi_sigmoid(sigmas[i, :], L)

    final_cov = a_np.zeros([dim, 1])                                                # (2L, 2L)
    for i in num_sigmas:
        diff = quasi_sigmoid(sigmas[i, :], L)-final_mean
        final_cov += Wc[i] * a_np.matmul(diff, diff.T)

    final_mean_upper = final_mean[:L]
    final_mean_lower = final_mean[L:]
    final_cross_cov = final_cov[:L, L:]
    A = a_np.linalg.inv(W)
    first_term = a_np.trace(a_np.matmul(A, final_cross_cov.T))
    second_term = a_np.matmul(a_np.matmul(final_mean_upper.T, A), final_mean_lower)
    return first_term + second_term


def expectation_term_three(params, dist_mean_t_1, dist_cov_t_1, u_quad_t):
    """
    E[ Phi(theta_{t-1})^T W^{-1} Phi(theta_{t-1}) | X]

    :param params: [G, G_in, W, v]
    :param dist_mean_t_1: E[theta_{t-1} | X]
    :param dist_cov_t_1: Cov[theta_{t-1} | X]
    :param u_quad_t: u_t^2
    :return: expectation of the third term: E[ Phi(theta_{t-1})^T W^{-1} Phi(theta_{t-1}) | X]
    """
    G, G_in, W, v = params
    # First make some sigma points
    dim = dist_mean_t_1.shape[0]
    SigmaMaker = MerweScaledSigmaPoints(dim, alpha=.1, beta=2., kappa=0)
    sigmas = SigmaMaker.sigma_points(dist_mean_t_1, dist_cov_t_1)                           # (2L+1, L)
    num_sigmas = sigmas.shape[0]
    Wm, Wc = SigmaMaker.Wm, SigmaMaker.Wc                                                   # (2L+1, )  (2L+1, )
    linear_transform = a_np.matmul(G, sigmas.T) + a_np.matmul(G_in, u_quad_t.reshape([-1,1]))  # (L, 2L+1)
    sigmoid_transform = 1 / (1 + a_np.exp(linear_transform))                                # (L, 2L+1)
    trans_mean = a_np.sum(a_np.multiply(Wm, sigmoid_transform), axis=1)                     # (L, )
    trans_cov = a_np.zeros([dim, dim])                                                      # (L, L)
    for i in range(num_sigmas):
        diff = (sigmoid_transform[:, i] - trans_mean).reshape([-1, 1])
        delta = Wc[i] * a_np.matmul(diff, diff.T)
        trans_cov += delta

    first_term = a_np.trace(a_np.matmul(a_np.linalg.inv(W), trans_cov))
    second_term = a_np.matmul(a_np.matmul(trans_mean.T, a_np.linalg.inv(W)), trans_mean)

    return first_term + second_term


def expectation_term_four(dist_mean_t, dist_cov_t, p_t, r_t, K_t, T_t):
    """
    This term doesn't involve the calculation of any parameter, but the numerical result will be used later
    E[Psi(theta_t) | X]

    :param dist_mean_t:
    :param dist_cov_t:
    :param p_t: current asset price
    :param r_t: current risk free interest rate
    :param K_t: strike price, 1d vector # (N, )
    :param T_t: maturity time, 1d vector # (N, )
    :return: expectation of the fourth term: E[Psi(theta_t) | X]
    """
    K_t = K_t.reshape([1, -1])                                                                  # (1, N)
    T_t = T_t.reshape([1, -1])                                                                  # (1, N)

    # First make some sigma points
    dim = dist_mean_t.shape[0]
    average_vector = a_np.ones([1, dim]) / dim
    mean_v_t = a_np.average(dist_mean_t)
    print(mean_v_t)
    var_v_t = a_np.matmul(a_np.matmul(average_vector, dist_cov_t), average_vector.T)

    SigmaMaker = MerweScaledSigmaPoints(1, alpha=.1, beta=2., kappa=0)
    sigmas = SigmaMaker.sigma_points(mean_v_t, var_v_t)                                         # (3, 1)
    Wm, Wc = SigmaMaker.Wm, SigmaMaker.Wc                                                       # (3, )  (3, )

    # Black Scholes parallel computing
    print("Term 4")
    print(sigmas)
    divider = a_np.sqrt(a_np.matmul(sigmas, T_t))                                               # (3, topK)
    d_pls = (a_np.log(p_t / K_t) + a_np.matmul(r_t + sigmas / 2, T_t)) / divider                # (3, topK)
    d_mns = (a_np.log(p_t / K_t) + a_np.matmul(r_t - sigmas / 2, T_t)) / divider                # (3, topK)
    first_term = a_np.multiply(p_t, a_ss.norm.cdf(d_pls))                                       # (3, topK)
    second_term = a_np.multiply(a_np.multiply(K_t, a_np.exp(-r_t * T_t)), a_ss.norm.cdf(d_mns)) # (3, topK)
    BS_transform = first_term - second_term                                                     # (3, topK)
    
    trans_mean = a_np.dot(Wm, BS_transform) # (topK, )

    return trans_mean


def expectation_term_five(dist_mean_t, dist_cov_t, p_t, r_t, K_t, T_t):
    """
    This term doesn't involve the calculation of any parameter, but the numerical result will be used later
    E[Psi(theta_t)^2 | X]

    :param dist_mean_t:
    :param dist_cov_t:
    :param p_t: current asset price
    :param r_t: current risk free interest rate
    :param K_t: strike price, 1d vector # (N, )
    :param T_t: maturity time, 1d vector # (N, )
    :return: expectation of the fifth term: E[Psi(theta_t)^2 | X]
    """
    topK = K_t.shape[0]
    K_t = K_t.reshape([1, -1])  # (1, N)
    T_t = T_t.reshape([1, -1])  # (1, N)

    # First make some sigma points
    dim = dist_mean_t.shape[0]
    average_vector = a_np.ones([1, dim]) / dim
    mean_v_t = a_np.average(dist_mean_t)
    var_v_t = a_np.matmul(a_np.matmul(average_vector, dist_cov_t), average_vector.T)

    SigmaMaker = MerweScaledSigmaPoints(1, alpha=.1, beta=2., kappa=0)
    sigmas = SigmaMaker.sigma_points(mean_v_t, var_v_t)                                         # (3, 1)
    Wm, Wc = SigmaMaker.Wm, SigmaMaker.Wc                                                       # (3, )  (3, )

    # Black Scholes parallel computing
    print("Term 5")
    print(sigmas)
    divider = a_np.sqrt(a_np.matmul(sigmas, T_t))                                               # (3, topK)
    d_pls = (a_np.log(p_t / K_t) + a_np.matmul(r_t + sigmas / 2, T_t)) / divider                # (3, topK)
    d_mns = (a_np.log(p_t / K_t) + a_np.matmul(r_t - sigmas / 2, T_t)) / divider                # (3, topK)
    first_term = a_np.multiply(p_t, a_ss.norm.cdf(d_pls))                                       # (3, topK)
    second_term = a_np.multiply(a_np.multiply(K_t, a_np.exp(-r_t * T_t)), a_ss.norm.cdf(d_mns)) # (3, topK)
    BS_transform = first_term - second_term                                                     # (3, topK)

    trans_mean = a_np.dot(Wm, BS_transform)  # (topK, )
    trans_cov = a_np.zeros([topK, topK])                                                        # (topK, topK) diagonal
    for i in range(3):
        diff = (BS_transform[i, :] - trans_mean).reshape([-1, 1])
        delta = Wc[i] * a_np.matmul(diff, diff.T)
        trans_cov += delta

    return a_np.sum(a_np.power(trans_mean, 2)) + a_np.trace(trans_cov)


def em_one(params,
           dist_mean_t, dist_mean_t_1,                          # Distributional Mean input
           dist_cov_t, dist_cov_t_1, dist_cross_t_t_1,          # Distributional Covariance input
           u_quad_t,                                            # Stock wise input
           p_t, r_t, K_t, T_t, y_t):                            # Option wise input
    """
    EM algorithm in one time step

    :param params:              [G, G_in, W, v]
    :param dist_mean_t:         E[theta_t | X]
    :param dist_mean_t_1:       E[theta_{t-1} | X]
    :param dist_cov_t:          Cov[theta_t | X]
    :param dist_cov_t_1:        Cov[theta_{t-1} | X]
    :param dist_cross_t_t_1:    P_{t,t-1}^*
    :param u_quad_t:            u_t^2
    :param p_t:                 current asset price
    :param r_t:                 current risk free interest rate
    :param K_t:                 strike price, 1d vector  # (N, )
    :param T_t:                 maturity time, 1d vector # (N, )
    :param y_t:                 option prices, 1d vector # (N, )
    :return: EM loss generated in one time step
    """
    G, G_in, W, v = params
    k = W.shape[0]  # dimension of state
    n_t = K_t.shape[0]  # topK of option observations

    # First line: variance part
    variance_part = -1/2*a_np.log(a_np.linalg.det(W)) - n_t/2*a_np.log(v)

    # Second Line: dynamic part
    # Inside bracket components:
    dyn_component_one = expectation_term_one(params, dist_mean_t, dist_cov_t)
    
#     dyn_component_two = -2*expectation_term_two(params, dist_mean_t, dist_mean_t_1,
#                                                 dist_cov_t, dist_cov_t_1, dist_cross_t_t_1,
#                                                 u_quad_t) # 别忘了有第二项！！
    
    dyn_component_three = expectation_term_three(params, dist_mean_t_1, dist_cov_t_1, u_quad_t)
    dynamic_part = -1/2*(dyn_component_one+dyn_component_three) 

    # Third Line: observation part
    # Inside bracket components:
    obs_component_one = a_np.sum(a_np.power(y_t, 2))
    obs_component_two = -2*a_np.sum(a_np.multiply(y_t, expectation_term_four(dist_mean_t, dist_cov_t,
                                                                             p_t, r_t, K_t, T_t)))
    obs_component_three = expectation_term_five(dist_mean_t, dist_cov_t, p_t, r_t, K_t, T_t)
    observation_part = -1/(2*v) * (obs_component_one+obs_component_two+obs_component_three)

    # Fourth Line: constant part (can be neglected)
    constant_part = log(2*pi)*(k+n_t/2)

    return variance_part+dynamic_part+observation_part+constant_part


def loss(params,
         dist_mean_lis, dist_cov_lis, dist_cross_lis,
         u_quad_lis,
         p_lis, r_lis, K_lis, T_lis, y_lis,
         reg=0.1):
    """

    :param params:          [G, G_in, W, v]
    :param dist_mean_lis:   E[theta_t | X] for all t
    :param dist_cov_lis:    Cov[theta_t | X] for all t
    :param dist_cross_lis:  P_{t,t-1}^* for all t
    :param u_quad_lis:      u_t^2
    :param p_lis:           current asset price
    :param r_lis:           current risk free interest rate
    :param K_lis:           strike price, 1d vector  # (N, )
    :param T_lis:           maturity time, 1d vector # (N, )
    :param y_lis:           option prices, 1d vector # (N, )
    :param reg:             regularization parameter for LASSO loss
    :return:
    """
    G, G_in, W, v = params

    # EM loss
    total_em_loss = 0
    n = len(p_lis)
    for i in range(n):
        total_em_loss += em_one(params,
                                dist_mean_t=dist_mean_lis[i+1], dist_mean_t_1=dist_mean_lis[i],
                                dist_cov_t=dist_cov_lis[i+1], dist_cov_t_1=dist_cov_lis[i],
                                dist_cross_t_t_1=dist_cross_lis[i],
                                u_quad_t=u_quad_lis[i],
                                p_t=p_lis[i], r_t=r_lis[i], K_t=K_lis[i], T_t=T_lis[i], y_t=y_lis[i])

    # LASSO
    lasso_loss = reg*(a_np.sum(a_np.abs(G)) + a_np.sum(a_np.abs(G_in)))
    return lasso_loss - total_em_loss


In [36]:
theta_dim = 2
u_dim = 1
connectivity = 0.5
spectral_radius = 0.9


# initialize ESN dynamics
ESN = EchoState(theta_dim, u_dim, connectivity, spectral_radius)

dt=1
points = MerweScaledSigmaPoints(theta_dim, alpha=.001, beta=2., kappa=0)
ukf = UnscentedKalmanFilter(dim_x=ESN.theta_dim, dim_z=topK, dt=1, fx=fx, hx=hx, points=points)
ukf.x = np.ones(ESN.theta_dim) # initial state
ukf.P *= 0.01 # initial uncertainty
v = 0.01
ukf.R = np.diag([v]*topK) # 1 standard
ukf.Q = np.diag([1]*theta_dim)


In [37]:
ms = [ukf.x.copy()]
Cs = [ukf.P.copy()]
u_quad_s = []
ps = []
rs = []
ys = []
Ks = []
us = []
Ts = []

all_days = sorted(list(data.keys()))
for day in all_days:
    # get observable data
    ## Stock Wise and market wise data
    p_t = data[day][0]["spotclose"].values[0]
    r_t = (data[day][1] + 1)**(1/260) - 1
    u_t = data[day][2].reshape([-1])

    ## Option wise data
    y_t = data[day][0]["close2"].values
    K_t = data[day][0]["strike"].values
    today = [d.date() for d in data[day][0]["date"]]
    expireday = [d.date() for d in data[day][0]["expirydate"]]
    T_t = np.busday_count(today, expireday)
    
    
    ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, u_quad_now=np.power(u_t,2))
    ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
    ms.append(ukf.x_post.copy())
    Cs.append(ukf.P_post.copy())
    u_quad_s.append(np.power(u_t,2))
    us.append(u_t)
    ps.append(p_t)
    rs.append(r_t)
    ys.append(y_t)
    Ks.append(K_t)
    Ts.append(T_t)

ms = np.array(ms)
Cs = np.array(Cs)
ms_s, Cs_s, _, cvs_s = ukf.rts_smoother(ms, Cs, fx=ESNf, G=ESN.G, G_in=ESN.G_in, u_s=us)

In [45]:
import warnings
warnings.simplefilter('error')

thing = jacobian(loss)
params = [ESN.G, ESN.G_in, ukf.Q, v]
print(loss(params,
      ms_s, Cs_s, cvs_s, 
      u_quad_s, 
      ps, rs, Ks, Ts, ys, 
      reg=0.01))
thing(params,
      ms_s, Cs_s, cvs_s, 
      u_quad_s, 
      ps, rs, Ks, Ts, ys, 
      reg=0.01)


-72.45765071341573
Term 4
[[-72.45765071]
 [-72.38693967]
 [-72.52836176]]


RuntimeWarning: invalid value encountered in sqrt

In [57]:
np.var(us)

0.0003154667370077618

In [62]:
sigmoid(-8)

0.0003353501304664781

In [3]:
from autograd import grad

thing = grad(loss)
thing

<function autograd.wrap_util.unary_to_nary.<locals>.nary_operator.<locals>.nary_f(*args, **kwargs)>

In [57]:
a = np.array([1,2,3])
b = np.array([[1,2,3,4,5],
              [2,4,6,8,10],
              [3,6,9,12,15]])
print(a.shape)
print(b.shape)
np.dot(a,b)


(3,)
(3, 5)


array([14, 28, 42, 56, 70])